# Here we look at (extended) dynamic mode composition, and its relation to Koopman

In [2]:
import numpy as np
import matplotlib.pyplot as plt

### The first toy example is taken from Clarence Rowley's talk
#### https://ipam.wistia.com/medias/sl5hs1srxf (~21:30)
We start with the following $z=F(z)$ dynamics
$$
\begin{bmatrix} z_1 \\
                z_2 
                \end{bmatrix}
=
\begin{bmatrix} \lambda z_1 \\
               \mu z_2 + (\lambda^2 - \mu)cz_1^2
               \end{bmatrix}
$$
The Koopman operator $U$ is then defined on observables by $U\phi = \phi \circ F$. \
The eigenvalues of $U$ are $\lambda$ and $\mu$ and their respective eigenfunctions (eigenobservables) are 
$$ \phi_\lambda(z) = z_1 $$
$$ \phi_\mu(z) = z_2 - cz_1^2 $$
We can use DMD to recover the eigenvalues.

We start with a dataset of points $Z = \begin{bmatrix} z^{(1)} z^{(2)} \dots z^{(n-1)} \end{bmatrix}$, and let $Z' = F(Z) = \begin{bmatrix} z^{(2)} z^{(3)} \dots z^{(n)} \end{bmatrix}$ \
We form the observation matrices $X = g(Z)$, $Y = g(Z')$ \
And then learn $A$ to relate these by $Y=AX$. \
It then turns out that the eigenvalues of $A$ are also eigenvalues of $U$!

In [28]:
# Dynamics (forward map) function
def F(z, lmbda=0.9, mu=0.5, c=1):
    z1,z2 = z
    newz = [lmbda*z1, mu*z2 + (lmbda**2 - mu)*c*z1**2]
    return np.array(newz)
def obs(z,obs_dct):
    observations = [g(z) for g in obs_dct]
    return np.array(observations)
obs_dct1 = [ 
    lambda z: z[0], 
    lambda z: z[1] 
]
obs_dct2 = [
    lambda z: z[0],
    lambda z: z[1],
    lambda z: z[0]**2
]
obs_dct3 = [
    lambda z: z[0],
    lambda z: z[1],
    lambda z: z[1]**2
]
Z = np.array([[-5,5],[-1,1],[1,1],[5,5]]).T # data points
# But different datasets give different results! (sometimes better, sometimes worst)
# Z = np.array([[-5,-5],[-1,-1],[-1,1],[-5,5]]).T
# Z = np.array([[-5,-5],[-1,-1],[-1,1],[-5,5]]).%

In [29]:
# First we test the linear case (c=0), which looks like
# F = [0.9   0]
#     [0.  0.5]
obs_dct = obs_dct1 # choose dictionary of observables
FZ = np.apply_along_axis(lambda z: F(z,c=0),0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print("Eigenvalues discovered", D) # sure enough we recover the eigenvalues (but other datasets may fail!)

Eigenvalues discovered [0.9 0.5]


In [30]:
# Then we test with c=1 (doesn't work because observables (z1,z2) not rich enough!)
obs_dct = obs_dct1 # choose dictionary of observables
FZ = np.apply_along_axis(F,0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print(D)

[0.9        2.00230769]


In [31]:
# Now we try with 2nd dictionary of observables (z1,z2,z1^2)
obs_dct = obs_dct2 # choose dictionary of observables
FZ = np.apply_along_axis(F,0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print(D)

[0.9  0.5  0.81]


In [32]:
# Now we try with 3rd dictionary of observables (z1,z2,z2^2) -- z2^2 is  not good enough to recover 2nd eval
obs_dct = obs_dct3 # choose dictionary of observables
FZ = np.apply_along_axis(F,0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print(D)

[0.9        0.82205673 4.76704327]


### Our second toy example is a Linear system : example 4.1 in the EDMD paper
If $ x_{n+1} = Tx_n $, and that $T$ is diagonalizable with eigenvector/value pairs $(v_i,\lambda_i)$ such that $Tv_i = \lambda_iv_i$.  
We let $w_i$ be the Riesz dual basis to $v_i$, that is $\langle w_i, v_j \rangle = \delta_{ij}$.  
Surprisingly (to me at least), $w_i$ are also eigenvectors of the adjoint of $T$ with the same eigenvalues, that is $T^* w_i = \lambda_i w_i$.  
The eigenfunctions of the Koopman operator are then
$$ \phi_{n_1,\dots,n_N}(x) = \prod_{i=1}^N \langle w_i, x \rangle^{n_i}$$

In the paper, they resolve everything into an orthonormal basis (and for eg. write $w_i^* v_j = \delta_{ij}$ and that $w_i$'s are left eigenvectors of $J$ (the matrix representation of $T$ in that orthonormal basis). I prefer the abstract notation, independent of a basis choice. This is also what is used in the Applied Koopmanism paper.

In [33]:
from scipy.special import eval_hermite

In [146]:
# Dynamics
A = np.array([[0.9, -0.1],
              [0.,   0.8]])

# Observations
def create_hermite(n,m):
    return lambda x,y: eval_hermite(n,x) * eval_hermite(m,y)
deg = 4
obs_dct = [create_hermite(i,j) for i in range(deg) for j in range(deg)]
def obs(z,obs_dct):
    x,y = z
    observations = [g(x,y) for g in obs_dct]
    return np.array(observations)

In [148]:
# Generate random data
Z = np.random.randn(2,100)
FZ = A@Z

# Evaluate the observables
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# Fit the Koopman matrix and find it's eigenvalues
K = Y @ np.linalg.pinv(X)
D,V = np.linalg.eig(K)
D

array([1.        +0.j        , 0.28984258+0.j        ,
       0.3890866 +0.09585266j, 0.3890866 -0.09585266j,
       0.50305358+0.10293423j, 0.50305358-0.10293423j,
       0.81      +0.j        , 0.9       +0.j        ,
       0.72      +0.j        , 0.66437998+0.j        ,
       0.64      +0.j        , 0.8       +0.j        ,
       0.729     +0.j        , 0.648     +0.j        ,
       0.576     +0.j        , 0.512     +0.j        ])

#### We see that we find the same top eigenvalues as in the EDMD paper
![edmd_eigenvalues](edmd_linearsys_eigenvalues.png)

I'm not sure how to explain the complex eigenvalues (I think Koopman is a positive self-adjoint operator and hence should only have real positive eigenvalues). Might be from numerical errors/approximations. Here's part of the spectrum found in the paper. Still not sure how to interpret it.
![edmd_spectrum](edmd_fig4_spectrum.png)